In [1]:
import os, sys
sys.path.insert(1, '../')

from importlib import reload
from tqdm import tqdm
from time import time

import numpy as np
import pandas as pd

import utils

In [2]:
native_df = pd.read_csv('Native_allchunks_test_tpu_model.csv', index_col=0)
native_df

,sequence,measured,predicted
index,,,
0,TGCATTTTTTTCACATCAAAAAAAAAAAAAAAAAAAAAAACTAGCA...,11.825806,11.353749
1,TGCATTTTTTTCACATCAAAAAAAAAAAAAAAATTTAAAGAATTTC...,11.412778,9.643128
2,TGCATTTTTTTCACATCAAAAAAAAGGACGTTCCACACTTGGCTGC...,10.623805,11.864820
3,TGCATTTTTTTCACATCAAAATCCGTGTAATACCAAGTCATCCTCA...,7.703389,8.650239
4,TGCATTTTTTTCACATCAGAAAAAAAGTATAATGTCAGATTTTTGT...,11.397473,13.881278
...,...,...,...
61145,TGCATTTTTTTCACATCAGAAAAAAAAAAAAAAAAAAAAAAAGAAA...,14.220082,12.520042
61146,TGCATTTTTTTCACATCAGAAAAAAAAAAAAGAGGAGACTTGAAGA...,6.978840,7.298204
61147,TGCATTTTTTTCACATCAGAAAAAAAAAAGGGATTGGGCGGTAATG...,3.917299,8.096985


In [10]:
reload(conc)
reload(utils)
n_trial = 500
qs = [0.5, 0.25, 0.75]
ns = np.logspace(np.log10(50), np.log10(1000), num=10, base=10).astype(int)
alpha = 0.1
delta = 0.09
theta_grid = np.arange(0, 20, 0.1)
c = 0.75
n_train = 0
save = True
save_fname = 'native-010423-delta{}.csv'.format(delta)

y_all = native_df.measured.to_numpy()
f_all = native_df.predicted.to_numpy()
f_all = 20 * (f_all - np.min(f_all)) / (np.max(f_all) - np.min(f_all))

columns = ['q', 'n', 'width', 'covered', 'estimator', 'lower', 'upper']
results = []
for q_idx, q in enumerate(qs):
    for n_idx, n in enumerate(ns):
        for _ in tqdm(range(n_trial)):
            
            ci_pp, ci_cl, cov_pp, cov_cl  = utils.trial(
                y_all, f_all, q, n, alpha, delta, theta_grid, c=c, n_train=n_train
            )
            results.append([q, n, ci_pp[1] - ci_pp[0], int(cov_pp), 'prediction-powered', ci_pp[0], ci_pp[1]])
            results.append([q, n, ci_cl[1] - ci_cl[0], int(cov_cl), 'classical', ci_cl[0], ci_cl[1]])
            
        df = pd.DataFrame(results, columns=columns)
        if save:
            df.to_csv(save_fname)
        
        print('n = {}, q = {}'.format(n, q))
        print('  avg. classical width: {:.2f}, avg. PP width: {:.2f}'.format(
            df.loc[(df.estimator == 'classical') & (df.n == n) & (df.q == q)].width.mean(),
            df.loc[(df.estimator == 'prediction-powered') & (df.n == n) & (df.q == q)].width.mean()
        ))
        print('  avg. classical cov: {:.2f}, avg. PP cov: {:.2f}'.format(
            df.loc[(df.estimator == 'classical') & (df.n == n) & (df.q == q)].covered.mean(),
            df.loc[(df.estimator == 'prediction-powered') & (df.n == n) & (df.q == q)].covered.mean()
        ))

100%|██████████| 500/500 [29:41<00:00,  3.56s/it]


n = 49, q = 0.5
  avg. classical width: 3.64, avg. PP width: 3.45
  avg. classical cov: 0.90, avg. PP cov: 1.00


100%|██████████| 500/500 [29:54<00:00,  3.59s/it]


n = 69, q = 0.5
  avg. classical width: 3.09, avg. PP width: 2.70
  avg. classical cov: 0.92, avg. PP cov: 1.00


100%|██████████| 500/500 [31:41<00:00,  3.80s/it]


...

n = 513, q = 0.75
  avg. classical width: 1.24, avg. PP width: 1.13
  avg. classical cov: 0.89, avg. PP cov: 0.97


100%|██████████| 500/500 [1:15:01<00:00,  9.00s/it]


n = 716, q = 0.75
  avg. classical width: 1.04, avg. PP width: 0.94
  avg. classical cov: 0.91, avg. PP cov: 0.97


100%|██████████| 500/500 [1:32:14<00:00, 11.07s/it]

n = 1000, q = 0.75
  avg. classical width: 0.88, avg. PP width: 0.81
  avg. classical cov: 0.90, avg. PP cov: 0.97
